Repeatノード

In [1]:
import numpy as np

In [5]:
D, N = 8, 7
x = np.random.randn(1, D)
y = np.repeat(x, N, axis=0) # forward

In [11]:
# 仮の勾配
dy = np.random.randn(N, D)
dx = np.sum(dy, axis=0, keepdims=True) # backward
dx

array([[-0.79664506, -1.09459353, -1.48867492,  1.63281182, -1.88013995,
         5.28855963, -2.54853688,  2.10819087]])

Sumノード

In [13]:
D, N = 8, 7
x = np.random.randn(N, D)
y = np.sum(x, axis=0, keepdims=True) # forward

In [14]:
dy = np.random.randn(1, D) # 仮の勾配
dx = np.repeat(dy, N, axis = 0)  # backward

MatMulノード

In [15]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)
        self.x = x
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        self.grads[0][...] = dW
        return dx

### 勾配の導出と逆伝播の実装

In [17]:
# Sigmoidレイヤ
class Sigmoid:
    def __init__(self):
        self.params, self.grads = [], []
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [18]:
# Affineレイヤ
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None
    
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dW = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...] = dW
        self.grads[1][...] = db
        return dx

### 1.3.6　重みの更新

In [19]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]